In [39]:
%matplotlib inline
from math import pi
import numpy as np
import matplotlib.pyplot as plt
from gtsam import *

# 2 Modeling Robot Motion

## 2.2 Creating a Factor Graph

In [23]:
def X(i):
    return symbol('x', i)

graph = NonlinearFactorGraph()
priorMean = Pose2(np.array([0.0, 0.0, 0.0]))
priorNoise = noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
priorFactor = PriorFactorPose2(X(1), priorPose, priorNoise)
graph.add(priorFactor)

odomMean = Pose2(np.array([2.0, 0.0, 0.0]))
odomNoise = noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(BetweenFactorPose2(X(1), X(2), odomMean, odomNoise))
graph.add(BetweenFactorPose2(X(2), X(3), odomMean, odomNoise))

In [24]:
print(graph)

size: 3

Factor 0: PriorFactor on x1
  prior mean: (0, 0, 0)
  noise model: diagonal sigmas[0.3; 0.3; 0.1];

Factor 1: BetweenFactor(x1,x2)
  measured: (2, 0, 0)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 2: BetweenFactor(x2,x3)
  measured: (2, 0, 0)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];




## 2.4 Non-linear Optimization in GTSAM

In [30]:
initial = Values()
initial.insert(X(1), Pose2(np.array([0.5, 0.0, 0.2])))
initial.insert(X(2), Pose2(np.array([2.3, 0.1, -0.2])))
initial.insert(X(3), Pose2(np.array([4.1, 0.1, 0.1])))
print("Initial Estimate:\n{}".format(initial))

result = LevenbergMarquardtOptimizer(graph, initial).optimize()
print("Final Result:\n{}".format(result))

Initial Estimate:
Values with 3 values:
Value x1: (gtsam::Pose2)
(0.496673327, 0.0498335554, 0.2)

Value x2: (gtsam::Pose2)
(2.29466402, -0.129899689, -0.2)

Value x3: (gtsam::Pose2)
(4.08817425, 0.30466264, 0.1)


Final Result:
Values with 3 values:
Value x1: (gtsam::Pose2)
(1.14850554e-15, -1.08248839e-15, -3.72019825e-16)

Value x2: (gtsam::Pose2)
(2, -2.26656227e-15, -5.18086301e-16)

Value x3: (gtsam::Pose2)
(4, -3.52656487e-15, -5.21041182e-16)




## 2.5 Full Posterior Inference

In [31]:
marginals = Marginals(graph, result)
print("x1 covariance:\n{}".format(marginals.marginalCovariance(X(1))))
print("x2 covariance:\n{}".format(marginals.marginalCovariance(X(2))))
print("x3 covariance:\n{}".format(marginals.marginalCovariance(X(3))))

x1 covariance:
[[ 9.00000000e-02 -3.32311241e-50  0.00000000e+00]
 [-3.32311241e-50  9.00000000e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e-02]]
x2 covariance:
[[1.30000000e-01 2.95802548e-18 1.47901274e-18]
 [2.95802548e-18 1.70000000e-01 2.00000000e-02]
 [1.47901274e-18 2.00000000e-02 2.00000000e-02]]
x3 covariance:
[[1.70000000e-01 1.87548749e-17 5.77831992e-18]
 [1.87548749e-17 3.70000000e-01 6.00000000e-02]
 [5.77831992e-18 6.00000000e-02 3.00000000e-02]]


# 3 Robot Localization

## 3.2 Defining Custom Factors

it seems that we can not define new gtsam factors in python currently,
see https://bitbucket.org/gtborg/gtsam/issues/251/simplified-way-to-define-new-types-of

In [40]:
# Create graph container and add factors to it
graph = gtsam.NonlinearFactorGraph()

# Add prior
# gaussian for prior
priorMean = gtsam.Pose2(0.0, 0.0, 0.0)  # prior at origin
priorNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
# add directly to graph
graph.add(gtsam.PriorFactorPose2(1, priorMean, priorNoise))

# Add odometry
# general noisemodel for odometry
odometryNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(gtsam.BetweenFactorPose2(
    1, 2, gtsam.Pose2(2.0, 0.0, 0.0), odometryNoise))
graph.add(gtsam.BetweenFactorPose2(
    2, 3, gtsam.Pose2(2.0, 0.0, pi / 2), odometryNoise))
graph.add(gtsam.BetweenFactorPose2(
    3, 4, gtsam.Pose2(2.0, 0.0, pi / 2), odometryNoise))
graph.add(gtsam.BetweenFactorPose2(
    4, 5, gtsam.Pose2(2.0, 0.0, pi / 2), odometryNoise))

# Add loop closure constraint
loopClosureNoise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
graph.add(gtsam.BetweenFactorPose2(5, 2, gtsam.Pose2(2.0, 0.0, pi / 2), loopClosureNoise))

In [41]:
graph

size: 6

Factor 0: PriorFactor on 1
  prior mean: (0, 0, 0)
  noise model: diagonal sigmas[0.3; 0.3; 0.1];

Factor 1: BetweenFactor(1,2)
  measured: (2, 0, 0)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 2: BetweenFactor(2,3)
  measured: (2, 0, 1.57079633)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 3: BetweenFactor(3,4)
  measured: (2, 0, 1.57079633)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 4: BetweenFactor(4,5)
  measured: (2, 0, 1.57079633)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 5: BetweenFactor(5,2)
  measured: (2, 0, 1.57079633)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];


In [42]:
# Initialize to noisy points
initialEstimate = gtsam.Values()
initialEstimate.insert(1, gtsam.Pose2(0.5, 0.0, 0.2))
initialEstimate.insert(2, gtsam.Pose2(2.3, 0.1, -0.2))
initialEstimate.insert(3, gtsam.Pose2(4.1, 0.1, pi / 2))
initialEstimate.insert(4, gtsam.Pose2(4.0, 2.0, pi))
initialEstimate.insert(5, gtsam.Pose2(2.1, 2.1, -pi / 2))

# Optimize using Levenberg-Marquardt optimization with an ordering from
# colamd
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initialEstimate)
result = optimizer.optimizeSafely()

In [43]:
graph.error(initialEstimate)

20.108558223490085

In [44]:
graph.error(result)

8.219132306212647e-18

In [45]:
help(findExampleDataFile)

Help on function findExampleDataFile in module gtsam.utils:

findExampleDataFile(name)
    Find the example data file specified by `name`.

